# Projecting COVID-19 Cases With One Step/Multi-Step ARIMA and VAR

### Import Python Modules

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from HackathonPackage import CovidModels
import datetime
import pickle
import json
import math

pd.set_option('display.max_columns', None) #setting to view all columns in the output


### Read dataset and display fields 

In [2]:
df = pd.read_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\CovidDataset.csv", engine='python', encoding = "utf-8-sig", parse_dates=['date'])
print(df.head())



iso_code continent      country state county  fips       date  total_cases  \
0      AFG      Asia  Afghanistan   N\A    N\A     0 2019-12-31          0.0   
1      AFG      Asia  Afghanistan   N\A    N\A     0 2020-01-01          0.0   
2      AFG      Asia  Afghanistan   N\A    N\A     0 2020-01-02          0.0   
3      AFG      Asia  Afghanistan   N\A    N\A     0 2020-01-03          0.0   
4      AFG      Asia  Afghanistan   N\A    N\A     0 2020-01-04          0.0   

   new_cases  total_deaths  new_deaths  total_tests  new_tests tests_units  \
0        0.0           0.0         0.0          NaN        NaN         NaN   
1        0.0           0.0         0.0          NaN        NaN         NaN   
2        0.0           0.0         0.0          NaN        NaN         NaN   
3        0.0           0.0         0.0          NaN        NaN         NaN   
4        0.0           0.0         0.0          NaN        NaN         NaN   

   stringency_index  population  population_density 

### Generate backtests for Countries

In [3]:
def GenerateComparisonCountry():
    counter = 0
    countryLen = len(df.country.unique())
    modelsDict = dict() #initialize a dicitionary to add dataframes to with key as the country name
    for country in df.country.unique():
        if counter < 500: #arbritray value soo all countries run
            #Generates a df template with date index to append predictions to
            timeDeltaObjectEnd = datetime.datetime.strptime("2020-07-14", "%Y-%m-%d")
            timeDeltaObjectStart = datetime.datetime.strptime("2020-07-01", "%Y-%m-%d")#df.index[len(df)-1] #gets last date in df var
            dateTimeDelta = timeDeltaObjectEnd - timeDeltaObjectStart
            dateTimeDeltaInt = dateTimeDelta.days
            compareDates = pd.date_range(start="2020-07-01", end="2020-07-14") #creates ranges of dates from start-end+delta
            modelsCompareDfAppend = pd.DataFrame(compareDates, columns=["date"]) #.set_index("date")
            modelsCompareDf = modelsCompareDfAppend
            #End of df template creation
            counter = counter + 1

            ModelGenerator = CovidModels.HackathonModels(df, country)
            countryDF = ModelGenerator.getcountrydf(fields=['date','total_cases']).asfreq('D').fillna(0)
            varDF = ModelGenerator.getcountrydf(fields=['date','total_cases','stringency_index']).asfreq('D').fillna(0)
        
            #countryDF = countryDF.loc[:'2020-06-30'] #Limits date for backtesting purposes
            #varDF = varDF.loc[:'2020-06-30'] #Limits date for backtesting purposes
            modelsCompareDf['country'] = country
            
            #Add in actuals to the modelsCompareDF
            try:
                backTestDF = countryDF.loc['2020-07-01':'2020-07-14']
                modelsCompareDf['actuals'] = backTestDF.total_cases.values.flatten()
            except:
                print('backtest error')
                modelsCompareDf['actuals'] = None
            
            countryDF = countryDF.loc[:'2020-06-30'] #Limits date for backtesting purposes
            varDF = varDF.loc[:'2020-06-30'] #Limits date for backtesting purposes
            
            #Run sarimax and add predictions to modelsCompareDf
            try:
                print('Running SARIMAX on ' + str(country))
                sarimaxDF = ModelGenerator.run_sarimax(countryDF,"2020-07-14")[0] #returns a DF of predicted cases and original 
                modelsCompareDf['sarimax'] = sarimaxDF.loc['2020-07-01':'2020-07-14'].new_cases_predict.values.flatten()
            except:
                print("sarimax error")
                modelsCompareDf['sarimax'] = None

            #Run SARIMAX with exogenous log fitted curve and add to modelsCompareDf
            try:
                print('Running SARIMAX exog on' + str(country))
                exogDF = ModelGenerator.getSigmoidFit(countryDF, "2020-07-14")
                sarimaxDFExog = ModelGenerator.run_sarimax_exog(countryDF,exogDF, '2020-07-14')[0]
                modelsCompareDf['sarimaxExog'] = sarimaxDFExog.loc['2020-07-01':'2020-07-14'].new_cases_predict.values.flatten()
            except:
                print("sarimax exog error")
                modelsCompareDf['sarimaxExog'] = None

            #Run VAR with endogenous stringency index and add to modelsCompareDf
            try:
                print('Running VAR on ' + str(country))
                varDF = ModelGenerator.run_VAR_MAVGStringency_tc(varDF, 13) #predicts 14 days forward
                modelsCompareDf['VAR'] = varDF.loc['2020-07-01':'2020-07-14'].total_cases_predict.values.flatten()                
            except:        
                print("VAR error")
                modelsCompareDf['VAR'] = None

            #Track which country we're on
            print(str(countryLen-counter) + " remaining")


            #Adds the stored predictions into the modelsDict dictionary
            modelsDict[country] = modelsCompareDf

            #Pickles the latest dictionary in case function breaks unexpectdly
            with open("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\inprogressdict.pk1", 'wb') as handle:
                pickle.dump(modelsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return modelsDict
        
dfCompare = GenerateComparisonCountry()
dfCompare

ing
Running SARIMAX on Ethiopia
Running SARIMAX exog onEthiopia
Running VAR on Ethiopia
total_cases
MovingAvgStringency
148 remaining
Running SARIMAX on Faeroe Islands
Running SARIMAX exog onFaeroe Islands
Running VAR on Faeroe Islands
All stringency values are null, exiting model
VAR error
147 remaining
Running SARIMAX on Falkland Islands
Running SARIMAX exog onFalkland Islands
Running VAR on Falkland Islands
All stringency values are null, exiting model
VAR error
146 remaining
Running SARIMAX on Fiji
Running SARIMAX exog onFiji
Running VAR on Fiji
total_cases
MovingAvgStringency
145 remaining
Running SARIMAX on Finland
Running SARIMAX exog onFinland
Running VAR on Finland
total_cases
MovingAvgStringency
144 remaining
Running SARIMAX on France
Running SARIMAX exog onFrance
Running VAR on France
total_cases
MovingAvgStringency
143 remaining
Running SARIMAX on French Polynesia
Running SARIMAX exog onFrench Polynesia
Running VAR on French Polynesia
All stringency values are null, exiting

2  2020-07-03  Turks and Caicos Islands     44.0   77.788787    77.551130   
 3  2020-07-04  Turks and Caicos Islands     44.0   89.349805    88.961720   
 4  2020-07-05  Turks and Caicos Islands     45.0  100.576689   100.006302   
 5  2020-07-06  Turks and Caicos Islands     47.0  111.479097   110.696618   
 6  2020-07-07  Turks and Caicos Islands     49.0  122.066406   121.044030   
 7  2020-07-08  Turks and Caicos Islands     49.0  132.347724   131.059539   
 8  2020-07-09  Turks and Caicos Islands     55.0  142.331894   140.753791   
 9  2020-07-10  Turks and Caicos Islands     55.0  152.027504   150.137090   
 10 2020-07-11  Turks and Caicos Islands     66.0  161.442894   159.219411   
 11 2020-07-12  Turks and Caicos Islands     66.0  170.586163   168.010408   
 12 2020-07-13  Turks and Caicos Islands     72.0  179.465175   176.519426   
 13 2020-07-14  Turks and Caicos Islands     72.0  188.087569   184.755511   
 
      VAR  
 0   None  
 1   None  
 2   None  
 3   None  
 4 

### Persist the country backtests data

In [5]:
import json
dfCompare

#Iterates through dictionary and creates a list of DFs
df_list = [v for k,v in dfCompare.items()] 

#Creates a combined DF from the combined list of DFs
combinedDF = pd.concat(df_list ,axis=0)

#pickles the combined dataframe, and save the combined dataframe to CSV to persist the data
combinedDF.to_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\BacktestDF.pk1")
combinedDF.to_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\BacktestDF.csv")
#newCombed = pd.read_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\BacktestDF.pk1")
#newCombed

#Example on how to load the inprogress pickle (represents the latest value in dictionary loop)
#with open("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\inprogressdict.pk1", 'rb') as handle:
#    pickle = pickle.load(handle)

#newCombed = pd.read_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\BacktestDF.pk1")
#newCombed


### Generate backtests for States

In [4]:
def GenerateComparisonState():
    counter = 0
    countryLen = len(df.state.unique())
    modelsDict = dict() #initialize a dicitionary to add dataframes to with key as the country name
    for country in df.state.unique(): #Should rename country variable to state but keeping for now
        if counter < 500: #arbritray value so all states run
            
            #Generates a df template with date index to append predictions to
            timeDeltaObjectEnd = datetime.datetime.strptime("2020-07-14", "%Y-%m-%d")
            timeDeltaObjectStart = datetime.datetime.strptime("2020-07-01", "%Y-%m-%d")#df.index[len(df)-1] #gets last date in df var
            dateTimeDelta = timeDeltaObjectEnd - timeDeltaObjectStart
            dateTimeDeltaInt = dateTimeDelta.days
            compareDates = pd.date_range(start="2020-07-01", end="2020-07-14") #creates ranges of dates from start-end+delta
            modelsCompareDfAppend = pd.DataFrame(compareDates, columns=["date"]) #.set_index("date")
            modelsCompareDf = modelsCompareDfAppend
            #End of df template creation

            #modelsCompareDf['country'] = country
            counter = counter + 1

            ModelGenerator = CovidModels.HackathonModels(df, country)

            countryDF = ModelGenerator.getstatedf(fields=['date','total_cases']).asfreq('D').fillna(0)
            #varDF = ModelGenerator.getcountrydf(fields=['date','total_cases','stringency_index']).asfreq('D').fillna(0) N/A model for states
        
            #countryDF = countryDF.loc[:'2020-06-30'] #Limits date for backtesting purposes
            #varDF = varDF.loc[:'2020-06-30'] #Limits date for backtesting purposes
            modelsCompareDf['state'] = country
            try:
                backTestDF = countryDF.loc['2020-07-01':'2020-07-14']
                modelsCompareDf['actuals'] = backTestDF.total_cases.values.flatten()
            except:
                print('backtest error')
                modelsCompareDf['actuals'] = None
            
            countryDF = countryDF.loc[:'2020-06-30'] #Limits date for backtesting purposes
            #varDF = varDF.loc[:'2020-06-30'] #Limits date for backtesting purposes N/A model for states
            
            try:
                print('Running SARIMAX on ' + str(country))
                sarimaxDF = ModelGenerator.run_sarimax(countryDF,"2020-07-14")[0] #returns a DF of predicted cases and original 
                modelsCompareDf['sarimax'] = sarimaxDF.loc['2020-07-01':'2020-07-14'].new_cases_predict.values.flatten()
            except:
                print("sarimax error")
                modelsCompareDf['sarimax'] = None

            try:
                print('Running SARIMAX exog on' + str(country))
                exogDF = ModelGenerator.getSigmoidFit(countryDF, "2020-07-14")
                sarimaxDFExog = ModelGenerator.run_sarimax_exog(countryDF,exogDF, '2020-07-14')[0]
                modelsCompareDf['sarimaxExog'] = sarimaxDFExog.loc['2020-07-01':'2020-07-14'].new_cases_predict.values.flatten()
            except:
                print("sarimax exog error")
                modelsCompareDf['sarimaxExog'] = None

#VAR model N/A for states
###
#            try:
#                print('Running VAR on ' + str(country))
#                varDF = ModelGenerator.run_VAR_MAVGStringency_tc(varDF, 13) #predicts 14 days forward
#                modelsCompareDf['VAR'] = varDF.loc['2020-07-01':'2020-07-14'].total_cases_predict.values.flatten()                
#            except:        
#                print("VAR error")
#                modelsCompareDf['VAR'] = None
###

            print(str(countryLen-counter) + " remaining")


            modelsDict[country] = modelsCompareDf
            #Pickles the latest dictionary in case function breaks unexpectdly
            with open("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\stateinprogressdict.pk1", 'wb') as handle:
                pickle.dump(modelsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return modelsDict
        
dfCompareState = GenerateComparisonState()
dfCompareState

backtest error
Running SARIMAX on N\A
sarimax error
Running SARIMAX exog onN\A
sarimax exog error
51 remaining
Running SARIMAX on AL
Running SARIMAX exog onAL
50 remaining
Running SARIMAX on AK
Running SARIMAX exog onAK
49 remaining
Running SARIMAX on AZ
Running SARIMAX exog onAZ
sarimax exog error
48 remaining
Running SARIMAX on AR
Running SARIMAX exog onAR
sarimax exog error
47 remaining
Running SARIMAX on CA
Running SARIMAX exog onCA
46 remaining
Running SARIMAX on CO
Running SARIMAX exog onCO
45 remaining
Running SARIMAX on CT
Running SARIMAX exog onCT
44 remaining
Running SARIMAX on DE
Running SARIMAX exog onDE
43 remaining
Running SARIMAX on DC
Running SARIMAX exog onDC
42 remaining
Running SARIMAX on FL
Running SARIMAX exog onFL
41 remaining
Running SARIMAX on GA
Running SARIMAX exog onGA
40 remaining
Running SARIMAX on HI
Running SARIMAX exog onHI
39 remaining
Running SARIMAX on ID
Running SARIMAX exog onID
38 remaining
Running SARIMAX on IL
Running SARIMAX exog onIL
37 remaini

E    21603  21441.202531  21305.827683,
 'NV':          date state  actuals       sarimax   sarimaxExog
 0  2020-07-01    NV    19101  19102.771648  19139.348377
 1  2020-07-02    NV    19732  19749.543295  19840.502457
 2  2020-07-03    NV    20717  20396.314943  20559.793226
 3  2020-07-04    NV    21575  21043.086591  21297.557796
 4  2020-07-05    NV    22418  21689.858238  22054.139515
 5  2020-07-06    NV    22909  22336.629886  22829.888081
 6  2020-07-07    NV    23785  22983.401534  23625.159659
 7  2020-07-08    NV    24301  23630.173181  24440.317000
 8  2020-07-09    NV    25033  24276.944829  25275.729563
 9  2020-07-10    NV    25906  24923.716477  26131.773632
 10 2020-07-11    NV    26838  25570.488124  27008.832448
 11 2020-07-12    NV    27683  26217.259772  27907.296331
 12 2020-07-13    NV    28515  26864.031420  28827.562812
 13 2020-07-14    NV    29619  27510.803067  29770.036762,
 'NH':          date state  actuals      sarimax  sarimaxExog
 0  2020-07-01    NH 

### Persist back test data for States

In [35]:
dfCompareState

#iterates through the dictionary returned from dfCompareState and returns as a list
df_list_state = [v for k,v in dfCompareState.items()] 

#Converts the list of DFs into a single combined DF
combinedDF_state = pd.concat(df_list_state,axis=0)

combinedDF_state.to_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\stateBacktestDF.pk1")
combinedDF_state.to_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\stateBacktestDF.csv")


#newCombed_state = pd.read_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\stateBacktestDF.pk1")
#newCombed
#Example on how to load the inprogress pickle (represents the latest value in dictionary loop)
#with open("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\stateinprogressdict.pk1", 'rb') as handle:
#    pickle_state = pickle.load(handle)



In [ ]:
#Note: Model backtested well for states, but this model relies on a curve being on a significant rise, which qualitatively may not be happening for the states as significantly as during the period from 7/1 through 7/14.
def getPolyFitStates():
    print(state)
    count = count + 1
    try:
        ModelGenerator = CovidModels.HackathonModels(df, state) #creates a HackathonModels object for FL
        stateDF_new = ModelGenerator.getstatedf(fields=['date','new_cases']).asfreq('D').fillna(0)
        stateDF_new = stateDF_new[:"2020-06-30"]
        exogDF = ModelGenerator.getPolynomialFit(stateDF_new, "2020-07-14") #4th degree polynomial fitting to historical data
        statePred = ModelGenerator.run_sarimax_exog(stateDF_new,exogDF, "2020-07-14")[0]

        #Gets a DF of total cases for state var
        stateDF_total = ModelGenerator.getstatedf(fields=['date','total_cases']).asfreq('D').fillna(0)
        
        #cleans data to set <7/14 
        statePred = statePred.fillna(0)
        statePred['new_cases_predict'][:'2020-06-30'] = 0
        statePred['new_cases_predict'] = statePred['new_cases_predict'] + statePred['new_cases']
        statePred['sarimax_exog_4dpoly'] = statePred.new_cases_predict.values.cumsum()
        statePred = statePred.merge(stateDF_total,how="left", right_index=True, left_index=True)
        statePred = statePred[['total_cases', 'sarimax_exog_4dpoly']]
        statePred["state"] = state 
        modelsDict[state] = statePred['2020-07-01':'2020-07-14']
        #statePred.to_csv('test.csv')
        #statePred[['cumsumpred', 'total_cases']].to_csv('test.csv')
        print(str(countryLen-count) + " remaining")
    except:
        modelsDict[state] = None

    return modelsDict


predictDict = getPolyFitStates()
predictDict

df_list = [v for k,v in predictDict.items()] 

#Creates a combined DF from the combined list of DFs
combinedDF = pd.concat(df_list ,axis=0)

combinedDF.to_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\ARIMA4dExog.csv")
combinedDF

### Loads the persisted backtest dataframes into a new variable. Persisting the backtests allows you to work on the data without reperforming the backtests.

In [14]:
import pandas as pd
#Loads the pickled dataframes for ARIMA, ARIMA with log exog, and VAr
newCombed_country = pd.read_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\BacktestDF.pk1")
#newCombed_state = pd.read_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\stateBacktestDF.pk1")

#Loads the CSV files for the LSTM model, this model was ran in a separate file
LSTM_country = pd.read_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\lstm_countries_without_early_stopping.csv")
#LSTM_state = pd.read_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\lstm_states_orig_model.csv")

#Mapping of prediction steps to dates for mergekey
stepCountMapping = {0:"2020-07-01",
    1:"2020-07-02",
    2:"2020-07-03",
    3:"2020-07-04",
    4:"2020-07-05",
    5:"2020-07-06",
    6:"2020-07-07",
    7:"2020-07-08",
    8:"2020-07-09",
    9:"2020-07-10",
    10:"2020-07-11",
    11:"2020-07-12",
    12:"2020-07-13",
    13:"2020-07-14"
    }

#Replaces the step count with dates for matching
LSTM_country.StepCount.replace(stepCountMapping, inplace=True)
LSTM_country.rename(columns={"prediction":"LSTM_Full"}, inplace=True)
#LSTM_state.StepCount.replace(stepCountMapping, inplace=True)
#LSTM_state.rename(columns={"prediction":"LSTM_Full"}, inplace=True)

#Creates a key to map predictions to other models
LSTM_country['mergekey'] = LSTM_country['StepCount'] + "-" + LSTM_country['region']
#LSTM_state['mergekey'] = LSTM_state['StepCount'] + "-" + LSTM_state['region']

#Removes uneeded columns
LSTM_country = LSTM_country[['LSTM_Full', 'mergekey']]
#LSTM_state = LSTM_state[['LSTM_Full', 'mergekey']]
#LSTM_state

#Helper function for calculating residual squared, to consider refactoring, but works for now
def ResidualsCalc(field1, field2):
    residualSquared = (field1 - field2) ** 2
    return residualSquared


#newCombed_state

### Calculates the RMSE, and selects the smallest RMSE out of the models performed

In [158]:
#Calculate RSME for states

newCombed_state['mergekey'] = newCombed_state['date'].dt.strftime("%Y-%m-%d") + "-" + newCombed_state['state']
newCombed_state = newCombed_state.merge(LSTM_state, right_on='mergekey', left_on='mergekey')

newCombed_state['sarimax_rsm'] = ResidualsCalc(newCombed_state.actuals,newCombed_state.sarimax) #((newCombed_state.actuals - newCombed_state.sarimax) ** 2)/14
newCombed_state['sarimaxExog_rsm'] = ResidualsCalc(newCombed_state.actuals, newCombed_state.sarimaxExog) #(newCombed_state.actuals - newCombed_state.sarimaxExog) ** 2
newCombed_state['LSTM_Full_rsm'] = ResidualsCalc(newCombed_state.actuals, newCombed_state.LSTM_Full) #

newCombed_state.to_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\RSMEStates_Validation.csv") #correct up to here



newCombedRSMState = newCombed_state.groupby('state')['sarimax_rsm', 'sarimaxExog_rsm', 'LSTM_Full_rsm'].sum() #sum of residuals squared

newCombedRSMState['sarimax_RMSE'] = (newCombedRSMState.sarimax_rsm/14) ** .5 #to 1/2 equals sqrt
newCombedRSMState['sarimaxExog_RMSE'] = (newCombedRSMState.sarimaxExog_rsm/14) ** .5 #to 1/2 equals sqrt
newCombedRSMState['LSTM_Full_RMSE'] = (newCombedRSMState.sarimaxExog_rsm/14) ** .5 #to 1/2 equals sqrt
#newCombedRSM['sarimax_RMSE'] = newCombedRSM.sarimax_rsm ** .5#to 1/2 equals sqrt
#newCombedRSM['sarimaxExog_RMSE'] = newCombedRSM.sarimaxExog_rsm ** .5 #to 1/2 equals sqrt

newCombedRSMState = newCombedRSMState[['sarimax_RMSE', 'sarimaxExog_RMSE', 'LSTM_Full_RMSE']]
newCombedRSMState.loc[newCombedRSMState.sarimax_RMSE == 0, 'sarimax_RMSE'] = 999999999 #arbritrary large value, since 0 means model failed
newCombedRSMState.loc[newCombedRSMState.sarimaxExog_RMSE == 0, 'sarimaxExog_RMSE'] = 999999999 #arbritrary large value, since 0 means model failed
newCombedRSMState.loc[newCombedRSMState.LSTM_Full_RMSE == 0, 'LSTM_Full_RMSE'] = 999999999

#Returns the smallest RMSE out of the models performed
newCombedRSMState['BestModel'] = newCombedRSMState.idxmin(axis=1)

#Sends data to CSV and pickles the data 
newCombedRSMState.to_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\RSMEStates.csv")
newCombedRSMState.to_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\stateBackTestRMSEResults.pk1")

In [15]:
#Calculate RMSE for countries
newCombed_country['mergekey'] = newCombed_country['date'].dt.strftime("%Y-%m-%d") + "-" + newCombed_country['country']
newCombed_country = newCombed_country.merge(LSTM_country, right_on='mergekey', left_on='mergekey')

#ResidualsCal function return the residual squared before taking a sum, averaging and sqrt.
newCombed_country['sarimax_rsm'] = ResidualsCalc(newCombed_country.actuals, newCombed_country.sarimax) #((newCombed_state.actuals - newCombed_state.sarimax) ** 2)/14
newCombed_country['sarimaxExog_rsm'] = ResidualsCalc(newCombed_country.actuals, newCombed_country.sarimaxExog) #(newCombed_state.actuals - newCombed_state.sarimaxExog) ** 2
newCombed_country['var_rsm'] = ResidualsCalc(newCombed_country.actuals, newCombed_country.VAR) #
newCombed_country['LSTM_Full_rsm'] = ResidualsCalc(newCombed_country.actuals, newCombed_country.LSTM_Full) #

newCombed_country.to_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\RSMECountries_Validation.csv") #correct up to here

#Takes the sum of the residuals squared
newCombedRSM = newCombed_country.groupby('country')['sarimax_rsm', 'sarimaxExog_rsm', 'var_rsm', 'LSTM_Full_rsm'].sum() #sum of residuals squared




newCombedRSM['sarimax_RMSE'] = (newCombedRSM.sarimax_rsm/14) ** .5 #to 1/2 equals sqrt
newCombedRSM['sarimaxExog_RMSE'] = (newCombedRSM.sarimaxExog_rsm/14) ** .5 #to 1/2 equals sqrt
newCombedRSM['var_RMSE'] = (newCombedRSM.var_rsm/14) ** .5
newCombedRSM['LSTM_Full_RMSE'] = (newCombedRSM.LSTM_Full_rsm/14) ** .5
#newCombedRSM['sarimax_RMSE'] = newCombedRSM.sarimax_rsm ** .5#to 1/2 equals sqrt
#newCombedRSM['sarimaxExog_RMSE'] = newCombedRSM.sarimaxExog_rsm ** .5 #to 1/2 equals sqrt

newCombedRSM = newCombedRSM[['sarimax_RMSE', 'sarimaxExog_RMSE', 'var_RMSE', 'LSTM_Full_RMSE']]
newCombedRSM.loc[newCombedRSM.sarimax_RMSE == 0, 'sarimax_RMSE'] = 999999999 #arbritrary large value, since 0 means model failed
newCombedRSM.loc[newCombedRSM.sarimaxExog_RMSE == 0, 'sarimaxExog_RMSE'] = 999999999 #arbritrary large value, since 0 means model failed
newCombedRSM.loc[newCombedRSM.var_RMSE == 0, 'var_RMSE'] = 999999999 #arbritrary large value, since 0 means model failed
newCombedRSM.loc[newCombedRSM.LSTM_Full_RMSE == 0, 'LSTM_Full_RMSE'] = 999999999 #arbritrary large value, since 0 means model failed


#Returns the smallest RMSE out of the models performed
newCombedRSM['BestModel'] = newCombedRSM.idxmin(axis=1)


#newCombedRSM
newCombedRSM.to_csv("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\RSMECountries.csv")
newCombedRSM.to_pickle("C:\\Users\\dylan.levine\\Development\\Covid Hackathon\\BacktestPersistedData\\countryBackTestRMSEResults.pk1")

In [128]:
newCombed_country['mergekey'] = newCombed_country['date'].dt.strftime("%Y-%m-%d") + "-" + newCombed_country['country']
newCombed_country = newCombed_country.merge(LSTM_country, right_on='mergekey', left_on='mergekey')
#newCombed_country.to_csv("test.csv")
newCombed_country

,date,country,actuals,sarimax,sarimaxExog,VAR,mergekey,LSTM_Full_x,sarimax_rsm,sarimaxExog_rsm,var_rsm,LSTM_Full_y,LSTM_Full_x,LSTM_Full_y
0,2020-07-01,Afghanistan,31517,3.149732e+04,31508.6,31501,2020-07-01-Afghanistan,3.446765e+04,387.385,69.9307,256,3.446765e+04,3.446765e+04,3.446765e+04
1,2020-07-02,Afghanistan,31836,3.170468e+04,31733,31765,2020-07-02-Afghanistan,3.578245e+04,17245.1,10608.8,5041,3.578245e+04,3.578245e+04,3.578245e+04
2,2020-07-03,Afghanistan,32022,3.192795e+04,31956.3,32054,2020-07-03-Afghanistan,3.696324e+04,8845.55,4320.55,1024,3.696324e+04,3.696324e+04,3.696324e+04
3,2020-07-04,Afghanistan,32324,3.216873e+04,32182.5,32324,2020-07-04-Afghanistan,3.828904e+04,24110.3,20034.6,0,3.828904e+04,3.828904e+04,3.828904e+04
4,2020-07-05,Afghanistan,32672,3.237010e+04,32381.2,32598,2020-07-05-Afghanistan,3.952303e+04,91145.9,84579,5476,3.952303e+04,3.952303e+04,3.952303e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,2020-07-10,World,1.22463e+07,1.191674e+07,1.19005e+07,1.22452e+07,2020-07-10-World,1.310346e+07,1.08597e+11,1.19548e+11,1.0816e+06,1.310346e+07,1.310346e+07,1.310346e+07
2936,2020-07-11,World,1.24763e+07,1.208147e+07,1.20621e+07,1.24623e+07,2020-07-11-World,1.333688e+07,1.55916e+11,1.71564e+11,1.98162e+08,1.333688e+07,1.333688e+07,1.333688e+07
2937,2020-07-12,World,1.26945e+07,1.224620e+07,1.22235e+07,1.26705e+07,2020-07-12-World,1.356634e+07,2.00984e+11,2.21891e+11,5.78114e+08,1.356634e+07,1.356634e+07,1.356634e+07
2938,2020-07-13,World,1.28898e+07,1.241092e+07,1.23845e+07,1.28687e+07,2020-07-13-World,1.380779e+07,2.29313e+11,2.55312e+11,4.4559e+08,1.380779e+07,1.380779e+07,1.380779e+07
